# Set-up

In [1]:
%pip install openai
%pip install langchain
%pip install python-dotenv

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Examples

Use openAI gpt4 model to generate a company named based on what it does

In [2]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.5)
text = "What would be a good company name for a company that makes colorful socks? Be creative and explain why this is a good name."
print(llm(text))



The name "Sock-Tacular" would be a good name for a company that makes colorful socks. This name implies that the socks made by this company are going to be something special and unique. It also has a fun, playful feel to it that would appeal to customers looking for something different.


Using prompt templates to organise prompts

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


# Chain

A chain is made up from 'links'.

Types:
1. LLMChain = PromptTemplate + LLM
2. Agent - handle user input, pick tools/toolkits, and respond
3. Tool - functions that the agent use to execute / take actions
4. Toolkit - a set of tools for certain tasks

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
## set up LLM
llm = OpenAI(temperature=0.9)

## set up prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

## create a chain (or, chain the action)
chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run("beautiful socks").strip()

# Agent
Agent will dynamically call chains based on user input. HOW?

It will use LLM to determine which actions to take, and in what order.

Actions include:
1. use a tool and 'observe' the output
2. return to user

Documents: 
- Agents: [supported agents and their specifications](https://python.langchain.com/en/latest/modules/agents/getting_started.html)
- Tools: [predefined tools and their specifications](https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html).

In [12]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

# First, let's load the language model we're going to use to control the agent. ()
llm = OpenAI(temperature=0, model_name="gpt4")

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [11]:
# Now let's test it out!

with get_openai_callback() as cb:
    output = agent.run("What is 32000 multiplied by 650 in the order of magnitude?")
    print(cb)
print(output)



> Entering new AgentExecutor chain...
 I need to calculate the product of these two numbers.
Action: Calculator
Action Input: 32000 * 650
Observation: Answer: 20800000
Thought: I need to determine the order of magnitude of this number.
Action: Calculator
Action Input: log(20800000)
Observation: Answer: 16.850463544671545
Thought: I now know the final answer.
Final Answer: The product of 32000 and 650 is in the order of magnitude of 10^16.

> Finished chain.
Tokens Used: 1054
	Prompt Tokens: 920
	Completion Tokens: 134
Successful Requests: 5
Total Cost (USD): $0.02108
The product of 32000 and 650 is in the order of magnitude of 10^16.


## Useful tools for Fermi-aid:

**python_repl** - a python shell
**wolfram-alpha** - need app id to query wolfram-alpha
**pal-math** - a LLM solving fully worded hard word math problem
**llm-math** - a LLMMath chain instance
**wikipedia** - use wikipedia python package to call MediaWiki API and parse results

Other: ChatGPT Plugin, Hugging Face


# Memory
Store information about the interactions. 
Need to imagine a chain/agent remembering key information over time - ST memory vs. LT memory

`ConversationChain` will remember all previous input/outputs and add them to the context.

In [14]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. My name is AI. What's your name?
